In [1]:
import numpy as np
import numpy.linalg as LA
import matplotlib.pyplot as plt
import pandas as pd
import time

from sklearn.datasets import make_moons
from scipy.sparse.linalg import cg
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.linear_model import SGDClassifier


from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
#from tqdm import tqdm_notebook as tqdm
import warnings
warnings.filterwarnings('ignore')

from lightonopu.opu import OPU
from lightonml.random_projections.opu import OPURandomMapping

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

from lightonml.datasets import MNIST
from sklearn.pipeline import Pipeline

pipeline_steps = []

In [2]:
random_state = np.random.RandomState(1234)

(X_train, y_train), (X_test, y_test) = MNIST()
X, y = np.concatenate([X_train, X_test]), np.concatenate([y_train, y_test])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10000,
                                                    random_state=42)

In [3]:
from lightonml.encoding.base import BinaryThresholdEncoder

encoder = BinaryThresholdEncoder()
print('Encoder threshold: ', encoder.threshold_enc)

pipeline_steps.append(('encoder', encoder))

Encoder threshold:  25


In [4]:

D = 10000 # number of random projections
opu = OPU(n_components=D, disable_pbar = False, verbose_level = 0)

random_mapping = OPURandomMapping(opu=opu)

pipeline_steps.append(('mapping', random_mapping))

In [5]:
from lightonml.encoding.base import NoDecoding

pipeline_steps.append(('decoding', NoDecoding()))



In [6]:
classifier = RidgeClassifier()

pipeline_steps.append(('classifier', classifier))

In [7]:
pipe = Pipeline(steps=pipeline_steps)

pipe.named_steps

{'classifier': RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                 max_iter=None, normalize=False, random_state=None,
                 solver='auto', tol=0.001),
 'decoding': NoDecoding(),
 'encoder': BinaryThresholdEncoder(greater_is_one=True, threshold_enc=25),
 'mapping': OPURandomMapping(opu=<lightonopu.opu.OPU object at 0x7f972d1b9cf8>)}

In [8]:
opu.open()

print('Fitting the model...')
pipe.fit(X_train, y_train)

Fitting the model...


Pipeline(memory=None,
         steps=[('encoder',
                 BinaryThresholdEncoder(greater_is_one=True, threshold_enc=25)),
                ('mapping',
                 OPURandomMapping(opu=<lightonopu.opu.OPU object at 0x7f972d1b9cf8>)),
                ('decoding', NoDecoding()),
                ('classifier',
                 RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True,
                                 fit_intercept=True, max_iter=None,
                                 normalize=False, random_state=None,
                                 solver='auto', tol=0.001))],
         verbose=False)

In [9]:
train_accuracy = pipe.score(X_train, y_train)
test_accuracy = pipe.score(X_test, y_test)

print('Train accuracy {:.2f}'.format(train_accuracy * 100))
print('Test accuracy {:.2f}'.format(test_accuracy * 100))


Train accuracy 98.31
Test accuracy 96.27


In [44]:
opu.close()

In [42]:
from tqdm import tqdm
def Kernel_OPU(X):
    n = X.shape[0]
    d = X.shape[1]
    K = np.zeros((n,n))
    for i in tqdm(range(n)):
        for j in range(n):
            K[i,j] = LA.norm(X[i,:])**2 * LA.norm(X[j,:])**2 + (X[i,:].reshape(1,-1)@(X[j,:].reshape(-1,1)))**2
    return K

In [43]:
K_train = Kernel_OPU(X_train)
K_test = Kernel_OPU(X_test)

  0%|          | 7/60000 [00:07<17:24:11,  1.04s/it]


KeyboardInterrupt: 

In [40]:
from sklearn.kernel_ridge import KernelRidge
clf = KernelRidge(alpha=1.0,kernel=K)
clf.fit(X, y) 

NameError: name 'K' is not defined

In [34]:
X_train.shape

(60000, 28, 28)